<a href="https://colab.research.google.com/github/AmanullaChishty/Pytorch/blob/main/Multimodel_Experiment_tracking_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ways to track ML experimnets
 - TensorBoard
 - Weights & Biases Experiment Tracking
 - MLFlow

## 0. Setup

In [1]:
import torch
import torchvision

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from torch import nn
from torchvision import transforms

In [3]:
!pip install -q torchinfo
from torchinfo import summary

In [4]:
!git clone https://github.com/mrdbourke/pytorch-deep-learning
!mv pytorch-deep-learning/going_modular .
!rm -rf pytorch-deep-learning
from going_modular.going_modular import data_setup, engine

Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4056, done.
remote: Total 4056 (delta 0), reused 0 (delta 0), pack-reused 4056
Receiving objects: 100% (4056/4056), 646.90 MiB | 32.33 MiB/s, done.
Resolving deltas: 100% (2371/2371), done.
Updating files: 100% (248/248), done.


In [5]:
def set_seeds(seed: int=42):
  torch.manual_seed(seed)

## 1. Get Data